In [1]:
#imports
import torch
from torch import nn
import pandas as pd
from torch.autograd import Variable
import numpy as np
from torch.optim.lr_scheduler import ExponentialLR
from tqdm import tqdm
from torch.utils.data import DataLoader
from IPython.utils import io
import warnings
import statistics
import torch.nn.functional as F

In [2]:
#read in optical flow rgb images file
images = pd.read_pickle('../data/images.pkl')
images = images.reset_index()
#read labels
labels = None
with open("../speedchallenge/data/train.txt") as f:
    #we drop the first label cause optical flow is two images
    labels = (list(map(float, f.read().splitlines()))[1:])
labels = np.array(labels)

In [10]:
#model class
class Small(nn.Module):
    
    def __init__(self, batch_size, dim):
        
        super(Small, self).__init__()
        #initialize parameters
        self.batch_size = batch_size
        self.dim = dim
        #decision layer
        self.fc1 = nn.Linear(dim, 2**9)
        self.fc2 = nn.Linear(2**9, 2**7)
        self.fc3 = nn.Linear(2**7, 1)
        #droput layers
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.4)
    
    def forward(self, x):
        
        #apply decision layers
        x = x.reshape(self.batch_size, -1).float()
        x = self.dropout1(x)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = F.relu(self.fc2(x))
        x = self.dropout3(x)
        x = self.fc3(x)
        return x

In [11]:
batch_size = 32
dim = 921600
n_train_examples = 100
n_val_examples = 32
n_epochs = 100
lr=1e-2

#initialize model
model = Small(batch_size=batch_size, dim=dim)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ExponentialLR(optimizer, gamma=0.99)
criterion = nn.MSELoss()

#use gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#create prediction model
model.to(device)

RuntimeError: CUDA out of memory. Tried to allocate 1.76 GiB (GPU 0; 11.91 GiB total capacity; 10.55 GiB already allocated; 607.25 MiB free; 10.55 GiB reserved in total by PyTorch)

In [ ]:
#function to create data loader
def get_loader(n, batch_size):
    #sample n images above sequence length index
    sample_images = images.sample(n).index
    vectors = list(images.iloc[idx].Image.reshape(3,480,640).numpy() for idx in tqdm(sample_images))
    targets = list(labels[idx] for idx in tqdm(sample_images))
    return vectors, targets

In [ ]:
#data loader classes
class MyTrainDataloader(torch.utils.data.Dataset):
    def __init__(self):
        self.data = list(zip(torch.tensor(x_train),torch.tensor(y_train)))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx][0], self.data[idx][1] 
    
class MyValDataLoader(torch.utils.data.Dataset):
    def __init__(self):
        self.data = list(zip(torch.tensor(x_val),torch.tensor(y_val)))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx][0], self.data[idx][1] 

In [ ]:
#create train and test data loaders
x_train, y_train = get_loader(n_train_examples, batch_size)
x_val, y_val = get_loader(n_val_examples, batch_size)

train_data = MyTrainDataloader()
train_loader = torch.utils.data.DataLoader(train_data,
                                           shuffle=True,
                                           num_workers=2,
                                           batch_size=batch_size,
                                           drop_last=True)

val_data = MyValDataLoader()
val_loader = torch.utils.data.DataLoader(val_data,
                                           shuffle=True,
                                           num_workers=2,
                                           batch_size=batch_size,
                                           drop_last=True)

In [ ]:
%%capture
#for switching model between train and eval without cell output
def train():
    with io.capture_output() as captured:
        model.train()

def test():
    with io.capture_output() as captured:
        model.eval()

In [ ]:
warnings.filterwarnings("ignore")

#track loss values
train_losses = []
val_losses = []

#begin training
for epoch in range(1, n_epochs + 1):
    
    #train
    train()
    train_loss = 0
    for batch in train_loader:
        x, y = batch[0], batch[1]
        optimizer.zero_grad()
        pred = model(torch.tensor(x).cuda())
        loss = criterion(pred.flatten(), torch.tensor(y).float().cuda())
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()
    scheduler.step()
    
    #validate
    if epoch%1 == 0:
        test()
        val_loss = 0
        for batch in test_loader:
            x, y = batch[0], batch[1]
            pred = model(torch.tensor(x).cuda())
            loss = criterion(pred.flatten(), y.float().cuda())
            val_loss+=loss.item()
        
        #print progress
        print('Epoch: {}/{}.............'.format(epoch, n_epochs))
        print("Train MSE: {:.4f}".format(train_loss/len(train_loader)))
        print("Val MSE: {:.4f}".format(val_loss/len(val_loader)), "\n")
        
        #track loss
        train_losses.append(train_loss/len(train_loader))
        val_losses.append(val_loss/len(val_loader))